In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Hard Voting

In [2]:
from sklearn.datasets import make_moons   #importing make_moons dataset for demonstration
from sklearn.ensemble import RandomForestClassifier, VotingClassifier   # importing classifiers
from sklearn.linear_model import LogisticRegression   # importing logistic regression model
from sklearn.model_selection import train_test_split  # importing train_test_split for data splitting
from sklearn.svm import SVC  # importing Support Vector Machine classifier

# Generating a synthetic dataset (make_moons)
X, y = make_moons(n_samples=500, noise=0.3, random_state=42)
# splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [3]:
# Creating a VotingClassifier with Logistic regression, Random Forest and Support Vector Machine.
voting_clf = VotingClassifier(               # if voting classifier is hard it uses counts to choose.
estimators=[
    ('lr', LogisticRegression(random_state=42)),
    ('rf', RandomForestClassifier(random_state=42)),
    ('svc', SVC(random_state=42))
])
# Fitting the VotingClassifier on the training data.
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(random_state=42))])

In [4]:
# Accessing the named_estimators_ attribute of the votingclassifier
voting_clf.named_estimators_

{'lr': LogisticRegression(random_state=42),
 'rf': RandomForestClassifier(random_state=42),
 'svc': SVC(random_state=42)}

In [5]:
for name, clf in voting_clf.named_estimators_.items():
    print(name, '=', clf.score(X_test, y_test))

lr = 0.864
rf = 0.896
svc = 0.896


In [6]:
voting_clf.predict(X_test[:1])

array([1])

In [7]:
y_test[:1]

array([1])

In [8]:
voting_clf.estimators_

[LogisticRegression(random_state=42),
 RandomForestClassifier(random_state=42),
 SVC(random_state=42)]

In [9]:
# Generating predictions for thr first sample of the test set using individual base estimators

In [10]:
[clf.predict(X_test[:1]) for clf in voting_clf.estimators_]

[array([1]), array([1]), array([0])]

In [11]:
voting_clf.score(X_test, y_test)

0.912

#Soft Voting

In [12]:
# configure the VotingClassifier for soft voting and enabling probability estimators for SVC

In [13]:
voting_clf.voting = "soft"
voting_clf.named_estimators['svc'].probability = True
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(probability=True, random_state=42))],
                 voting='soft')

In [14]:
voting_clf.score(X_test, y_test)

0.92

#Bagging and Pasting

#Bagging (Bootstrap Aggregating):
Uses the same data multiple times: Bagging involves creating multiple bootstrap samples (random samples with replacement) from the original dataset. These samples may contain duplicate instances.

Trains different parts of the same data with the same model: Each bootstrap sample is used to train a separate instance of the same model. The final prediction is then an average or majority vote of the predictions from each model.

Pasting
Uses it once and does not repeat: Pasting also involves creating multiple samples, but without replacement. Each instance is selected only once in each sample.

Trains different parts of the same data with the same model: Similar to bagging, pasting trains separate models on different samples.

So, in both bagging and pasting, different parts of the same data are used to train different models. The main difference lies in whether the sampling is done with or without replacement.

Bagging samples with replacement, while pasting samples without replacement. Both techniques aim to reduce overfitting and improve model generalization by introducing diversity through multiple models trained on different subsets of the data.

In [15]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [16]:
# Create a BaggingClassifier with DecisionTreeClassifier as the base estimator
bag_clf = BaggingClassifier(DecisionTreeClassifier(),
                           n_estimators=500,
                           max_samples=100,
                           n_jobs=-1,
                           random_state=42)

In [17]:
bag_clf.fit(X_train, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=100,
                  n_estimators=500, n_jobs=-1, random_state=42)

In [18]:
bag_clf.score(X_test, y_test)

0.904

#Out-of-Bag Evaluation

In [19]:
# Create a baggingClassifier with out-of-bag (OOB) score calculation
bag_clf = BaggingClassifier(DecisionTreeClassifier(),
                           n_estimators=500,
                           oob_score=True,
                           n_jobs=-1,
                           random_state=42)

In [20]:
bag_clf.fit(X_train, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=500,
                  n_jobs=-1, oob_score=True, random_state=42)

In [21]:
bag_clf.score(X_test, y_test)

0.92

In [22]:
bag_clf.oob_score_

0.896

#In bagging, on average, each base model is trained on about 63% of the original dataset. This is because, during bootstrap sampling, about 37% of the data points are not included in each bootstrap sample.

When you set the oob_score parameter to True in a bagging ensemble (e.g., BaggingClassifier or BaggingRegressor in scikit-learn), it indicates that you want to evaluate the model's performance on the out-of-bag (OOB) instances – the approximately 37% of data points that were not included in the bootstrap sample for each base model.

The OOB score serves as a validation metric without the need for a separate validation set. It helps estimate how well the ensemble is likely to generalize to unseen data. This is a useful feature of bagging, providing a built-in validation mechanism during the training process.

In [23]:
from sklearn.metrics import accuracy_score

In [24]:
y_pred = bag_clf.predict(X_test)

In [25]:
accuracy_score(y_test, y_pred)

0.92

#Random Forest

In [33]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators=500,
                                max_leaf_nodes=16,
                                n_jobs=-1,
                                random_state=42)

In [34]:
rnd_clf.fit(X_train, y_train)

RandomForestClassifier(max_leaf_nodes=16, n_estimators=500, n_jobs=-1,
                       random_state=42)

In [35]:
y_pred_rf = rnd_clf.predict(X_test)

In [36]:
bag_clf = BaggingClassifier(DecisionTreeClassifier(max_features='sqrt', max_leaf_nodes=16),
                           n_estimators=500,
                           n_jobs=-1,
                           random_state=42)

In [37]:
bag_clf.fit(X_train, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(max_features='sqrt',
                                                   max_leaf_nodes=16),
                  n_estimators=500, n_jobs=-1, random_state=42)

In [31]:
y_pred_bag = bag_clf.predict(X_test)

In [32]:
np.all(y_pred_bag == y_pred_rf)

np.True_

#Feature Importance

In [38]:
from sklearn.datasets import load_iris

In [42]:
# Loading the iris dataset as a DataFrame
iris = load_iris(as_frame=True)
rnd_clf = RandomForestClassifier(n_estimators=500, random_state=42)

In [40]:
rnd_clf.fit(iris.data, iris.target)

RandomForestClassifier(n_estimators=500, random_state=42)

In [41]:
for score, anme in zip(rnd_clf.feature_importances_, iris.data.columns):
    print(round(score, 2) * 100, '%', name)

11.0 % svc
2.0 % svc
44.0 % svc
42.0 % svc


#Boosting
#AdaBoosting

In [44]:
from sklearn.ensemble import AdaBoostClassifier
# Creating an AdaBoostClassifier with a base DesicionTreeClassifier
ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1),
    n_estimators=30,
    learning_rate=0.5,
    random_state=42
)

In [45]:
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=30, random_state=42)

In [46]:
ada_clf.score(X_test, y_test)

0.88

#Gradient Boosting

In [47]:
from sklearn.tree import DecisionTreeRegressor

In [48]:
np.random.seed(42)

In [49]:
X = np.random.rand(100, 1) - 0.5
y = 3 * X[:, 0] ** 2 + 0.05 * np.random.randn(100)

In [50]:
tree_reg1 = DecisionTreeRegressor(max_depth=2, random_state=42)

In [51]:
tree_reg1.fit(X, y)

DecisionTreeRegressor(max_depth=2, random_state=42)

In [52]:
y2 = y - tree_reg1.predict(X)

In [53]:
tree_reg2 = DecisionTreeRegressor(max_depth=2, random_state=43)

In [54]:
tree_reg2.fit(X, y2)

DecisionTreeRegressor(max_depth=2, random_state=43)

In [55]:
y3 = y2 - tree_reg1.predict(X)

In [56]:
tree_reg3 = DecisionTreeRegressor(max_depth=2, random_state=43)

In [57]:
tree_reg3.fit(X, y3)

DecisionTreeRegressor(max_depth=2, random_state=43)

In [58]:
X_new = np.array([[-0.4], [0.], [0.5]])
sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

array([-5.55111512e-17, -9.03979359e-02,  3.99129745e-02])

In [59]:
# Instead f writing the above manually, we can use GradientBoostingRegressor

In [71]:
from sklearn.ensemble import GradientBoostingRegressor
gbrt = GradientBoostingRegressor(max_depth=2,
                                n_estimators=3,
                                learning_rate=1.0,
                                random_state=42)

In [72]:
gbrt.fit(X, y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3,
                          random_state=42)

#As the learning rate increases, we reduce the number of estimators so that it is possible to arrive at a reach point

When the learning rate is increased, it means that each weak learner's contribution to the ensemble is stronger. To avoid overfitting and find an optimal trade-off between accuracy and generalization, it's common to reduce the number of estimators (trees) in the ensemble.

Here's a breakdown of the idea:

Low Learning Rate (e.g., 0.01 or 0.1):
Each tree has a smaller impact on the final prediction.
Requires a larger number of trees to fit the model well.
High Learning Rate (e.g., 0.5 or 1.0):
Each tree has a more substantial impact on the final prediction.
Requires a smaller number of trees to fit the model well.
By decreasing the number of trees as the learning rate increases, you can control the complexity of the overall model. This helps prevent overfitting and ensures that the boosting algorithm converges to a good solution. This strategy is often part of the hyperparameter tuning process in gradient boosting, where the learning rate and the number of estimators are tuned together.

In scikit-learn's GradientBoostingRegressor or GradientBoostingClassifier, you can achieve this by adjusting the n_estimators parameter based on your chosen learning rate.

In [73]:
gbrt_best = GradientBoostingRegressor(
    max_depth=2,
    learning_rate=0.05,
    n_estimators=500,
    n_iter_no_change=10,
    random_state=42)

In [74]:
gbrt_best.fit(X, y)

GradientBoostingRegressor(learning_rate=0.05, max_depth=2, n_estimators=500,
                          n_iter_no_change=10, random_state=42)

#AdaBoost (Adaptive Boosting):
Takes all the data and corrects errors: AdaBoost adapts by assigning different weights to instances in the training set. It focuses more on instances that were misclassified by the previous weak learners.

Global approach to correcting errors: The entire dataset is considered during each iteration, and weights are adjusted to emphasize correcting errors made on the entire dataset.

Gradient Boosting:
Selects and corrects errors through gradients: Gradient boosting builds a sequence of weak learners, and each learner corrects the errors of the combined ensemble of the previous learners. The emphasis is on minimizing the loss function by finding the gradient of the loss with respect to the predicted values.
Local approach to correcting errors: Each new weak learner in gradient boosting focuses on the errors made by the current ensemble, rather than adjusting weights for the entire dataset. This leads to a more localized and targeted correction of errors.

In [76]:
gbrt_best.n_estimators_

92

#StackingClassifier

In [65]:
from sklearn.ensemble import StackingClassifier

In [66]:
stacking_clf = StackingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=42)),
        ('rf', RandomForestClassifier(random_state=42)),
        ('svm', SVC(probability=True, random_state=42))
    ],
    final_estimator = RandomForestClassifier(random_state=43), # final estimator for blending estimators
    cv=5
)

In [67]:
stacking_clf.fit(X_train, y_train)

StackingClassifier(cv=5,
                   estimators=[('lr', LogisticRegression(random_state=42)),
                               ('rf', RandomForestClassifier(random_state=42)),
                               ('svm', SVC(probability=True, random_state=42))],
                   final_estimator=RandomForestClassifier(random_state=43))

In [77]:
stacking_clf.score(X_test, y_test)

0.928

#ExtraTreeClassifier is also Ensemble Model

#Mentor Hours

In [78]:
!kaggle datasets download -d aadeshkoirala/mnist-784

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'


In [79]:
!unzip /content/mnist-784.zip

unzip:  cannot find or open /content/mnist-784.zip, /content/mnist-784.zip.zip or /content/mnist-784.zip.ZIP.


In [80]:
df = pd.read_csv('/content/mnist_784.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/mnist_784.csv'

In [ ]:
df

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784,class
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
69996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
69997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
69998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5


In [ ]:
X = df.drop('class', axis=1)
y = df['class'].copy()

In [ ]:
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.2)

In [ ]:
X_train = X[:50000]
X_valid = X[10000:]
y_train = y[:50000]
y_valid = y[10000:]

In [ ]:
y_train_1 = (y_train == 1)
y_valid_1 = (y_valid == 1)

#Hard Voting

In [ ]:
voting_clf = VotingClassifier(
estimators=[
    ('lr', LogisticRegression(random_state=42)),
    ('rf', RandomForestClassifier(random_state=42)),
    ('svc', SVC(random_state=42))
])

voting_clf.fit(X_train, y_train_1)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(random_state=42))])

In [ ]:
voting_clf.named_estimators_

{'lr': LogisticRegression(random_state=42),
 'rf': RandomForestClassifier(random_state=42),
 'svc': SVC(random_state=42)}

In [ ]:
for name, clf in voting_clf.named_estimators_.items():
    print(name, '=', clf.score(X_valid, y_valid_1))

lr = 0.9936666666666667
rf = 0.9989166666666667
svc = 0.99815


In [ ]:
voting_clf.predict(X_valid[:1])

array([False])

In [ ]:
voting_clf.estimators_

[LogisticRegression(random_state=42),
 RandomForestClassifier(random_state=42),
 SVC(random_state=42)]

In [ ]:
[clf.predict(X_valid[:1]) for clf in voting_clf.estimators_]

[array([0]), array([0]), array([0])]

In [ ]:
voting_clf.score(X_valid, y_valid_1)

0.9982666666666666

#Soft Voting

In [ ]:
voting_clf.voting = "soft"
voting_clf.named_estimators['svc'].probability = True
voting_clf.fit(X_valid, y_valid_1)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(probability=True, random_state=42))],
                 voting='soft')

In [ ]:
voting_clf.score(X_valid, y_valid_1)

0.9985666666666667